In [2]:
#This scripts creates sqlite database in current script directory for all public data sources required
import requests, zipfile, io
import pandas as pd
import sqlite3 as sql

#Import functions from functions file
from functions import sql_create_table
from functions import zip_downloader
from functions import db_query
from functions import read_sql_string 

#MEPS has instructions for obtaining MEPS tables through R: https://github.com/HHS-AHRQ/MEPS/tree/master/R
#Python methods to open dat files are improperly formatted, so using rpy2 to run R in Python
#Install rpy2 and R libraries if needed
%pip install rpy2 
%load_ext rpy2.ipython
%conda install -c r r-essentials
from rpy2.robjects.packages import importr
utils = importr('utils')
utils.install_packages('foreign')
utils.install_packages('survey')
utils.install_packages('tidyverse')
utils.install_packages('readr')
utils.install_packages('devtools')
import rpy2.robjects as robjects
import rpy2.robjects.packages as packages
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.pandas2ri import py2rpy, rpy2py
from rpy2.robjects.conversion import localconverter

# #Load R libraries and install MEPS github packages. The '%R' allows you to run R in Python
%R library(foreign)
%R library(survey)
%R library(devtools)
%R library(tidyverse)
%R library(readr)
%R library(stringr)
%R install_github("e-mitchell/meps_r_pkg/MEPS")

utils.install_packages('devtools::install_github("e-mitchell/meps_r_pkg/MEPS")')

%R library(MEPS)

#TODO (low priority): Figure out how to convert r dataframe to pandas dataframe without the write to csv step... 
#File 206a is the 2018 version of the MEPS Prescribed Medications table. Each year/version, these tables are given a new file name. 
#Web scraping code in progress to pull the latest file name from the MEPS website. 
%R meps_prescriptions_2018 <- read_MEPS(year = 2018, type = 'DV')
%R meps_prescriptions_2018 <- read_MEPS(file = 'h206a')
%R write.csv(meps_prescriptions_2018, 'meps2018_prescribedmeds.csv')
#Read in MEPS as pandas dataframe, add to SQL db. Read in as strings to keep leading zeroes
MEPS_PrescribedMeds = pd.read_csv('meps2018_prescribedmeds.csv', dtype='str')
sql_create_table('MEPS_PrescribedMeds',MEPS_PrescribedMeds)

#File 209 is the 2018 version of the MEPS Patient Demographics file. 
%R meps_demographics_2018 <- read_MEPS(year = 2018, type = 'DV')
%R meps_demographics_2018 <- read_MEPS(file = 'h209')
%R write.csv(meps_demographics_2018, 'meps2018_patientdemographics.csv')
#Read in MEPS as pandas dataframe, add to SQL db
MEPS_PatientDemographics = pd.read_csv('meps2018_patientdemographics.csv')
sql_create_table('MEPS_PatientDemographics',MEPS_PatientDemographics)


# #TODO: Figure out way to handle different years...either by reading html table or manually entery of file name.
# z = zip_downloader('https://www.meps.ahrq.gov/mepsweb/data_files/pufs/h206adat.zip')


# #TODO:-1 from all colspecs data since python starts a 0 and r starts at 1
# col_names=['DUID', 'PID', 'DUPERSID', 'DRUGIDX', 'RXRECIDX', 'LINKIDX','PANEL', 'PURCHRD', 'RXBEGMM', 'RXBEGYRX', 'RXNAME',
#             'RXDRGNAM', 'RXNDC', 'RXQUANTY', 'RXFORM', 'RXFRMUNT','RXSTRENG', 'RXSTRUNT', 'RXDAYSUP', 'PHARTP1', 'PHARTP2',
#             'PHARTP3', 'PHARTP4', 'PHARTP5', 'PHARTP6', 'PHARTP7','PHARTP8', 'PHARTP9', 'RXFLG', 'IMPFLAG', 'PCIMPFLG',
#             'DIABEQUIP', 'INPCFLG', 'PREGCAT', 'TC1', 'TC1S1','TC1S1_1', 'TC1S1_2', 'TC1S2', 'TC1S2_1', 'TC1S3',
#             'TC1S3_1', 'TC2', 'TC2S1', 'TC2S1_1', 'TC2S1_2', 'TC2S2','TC3', 'TC3S1', 'TC3S1_1', 'RXSF18X', 'RXMR18X', 'RXMD18X',
#             'RXPV18X', 'RXVA18X', 'RXTR18X', 'RXOF18X', 'RXSL18X','RXWC18X', 'RXOT18X', 'RXOR18X', 'RXOU18X', 'RXXP18X',
#             'PERWT18F', 'VARSTR', 'VARPSU']
# MEPS = pd.read_fwf(z.open('H206A.dat'),header=None,names=col_names,converters={col:str for col in col_names},
#     colspecs=[(0,7),(7,10),(10,20),(20,33),(33,52),(52,68),(68,70),(70,71),(71,74),(74,78),(78,128),(128,188),(188,199),
#                 (199,206),(206,256),(256,306),(306,356),(356,406),(406,409),(409,412),(412,414),(414,416),(416,418),(418,420),(420,422),
#                 (422,424),(424,426),(426,428),(428,429),(429,430),(430,431),(431,432),(432,433),(433,436),(436,439),(439,442),(442,445),
#                 (445,448),(448,451),(451,454),(454,456),(456,458),(458,461),(461,464),(464,467),(467,470),(470,473),(473,476),(476,479),
#                 (479,482),(482,490),(490,498),(498,506),(506,514),(514,522),(522,529),(529,536),(536,543),(543,550),(550,558),(558,566),
#                 (566,573),(573,581),(581,593),(593,597),(597,None)])

# sql_create_table('MEPS',MEPS)


#deletes zip for MEPS
# del z


z = zip_downloader('https://download.nlm.nih.gov/rxnorm/RxNorm_full_prescribe_current.zip')

#moves RxNorm files to sqlite database by reading as dataframes
col_names = ['RXCUI','LAT','TS','LUI','STT','SUI','ISPREF','RXAUI','SAUI','SCUI','SDUI','SAB','TTY','CODE','STR','SRL','SUPPRESS','CVF','test']
RXNCONSO = pd.read_csv(z.open('rrf/RXNCONSO.RRF'),sep='|',header=None,dtype=object,names=col_names)
sql_create_table('RXNCONSO',RXNCONSO)

col_names = ['RXCUI1','RXAUI1','STYPE1','REL','RXCUI2','RXAUI2','STYPE2','RELA','RUI','SRUI','SAB','SL','DIR','RG','SUPPRESS','CVF','test']
RXNREL = pd.read_csv(z.open('rrf/RXNREL.RRF'),sep='|',dtype=object,header=None,names=col_names)
sql_create_table('RXNREL',RXNREL)

col_names = ['RXCUI','LUI','SUI','RXAUI','STYPE','CODE','ATUI','SATUI','ATN','SAB','ATV','SUPPRESS','CVF','test']
RXNSAT = pd.read_csv(z.open('rrf/RXNSAT.RRF'),sep='|',dtype=object,header=None,names=col_names)
sql_create_table('RXNSAT',RXNSAT)

#deletes RxNorm ZIP
del z

#TEST!!!!!!!!!!!!!!!! reads top line from MEPS data in database into dataframe
MEPS = db_query("Select count(*) AS records from MEPS limit 1")
print('DB table MEPS has {0} records'.format(MEPS['records'].iloc[0]))

RXNCONSO = db_query("Select count(*) AS records from RXNCONSO limit 1")
print('DB table RXNCONSO has {0} records'.format(RXNCONSO['records'].iloc[0]))

RXNREL = db_query("Select count(*) AS records from RXNREL limit 1")
print('DB table RXNREL has {0} records'.format(RXNREL['records'].iloc[0]))

RXNSAT = db_query("Select count(*) AS records from RXNSAT limit 1")
print('DB table RXNSAT has {0} records'.format(RXNSAT['records'].iloc[0]))


#TODO We could just execute a create table statement directly on the database, but this works and ported easily from joeys script
joey_query_str = read_sql_string('joey_query.sql')

joey_query = db_query(joey_query_str)
sql_create_table('joey_query', joey_query)


Note: you may need to restart the kernel to use updated packages.
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - r-essentials


The following NEW packages will be INSTALLED:

  _r-mutex           pkgs/r/win-64::_r-mutex-1.0.0-anacondar_1
  m2w64-bwidget      pkgs/msys2/win-64::m2w64-bwidget-1.9.10-2
  m2w64-bzip2        pkgs/msys2/win-64::m2w64-bzip2-1.0.6-6
  m2w64-expat        pkgs/msys2/win-64::m2w64-expat-2.1.1-2
  m2w64-fftw         pkgs/msys2/win-64::m2w64-fftw-3.3.4-6
  m2w64-flac         pkgs/msys2/win-64::m2w64-flac-1.3.1-3
  m2w64-gettext      pkgs/msys2/win-64::m2w64-gettext-0.19.7-2
  m2w64-gsl          pkgs/msys2/win-64::m2w64-gsl-2.1-2
  m2w64-libiconv     pkgs/msys2/win-64::m2w64-libiconv-1.14-6
  m2w64-libjpeg-tur~ pkgs/msys2/win-64::m2w64-libjpeg-turbo-1.4.2-3
  m2w64-libogg       pkgs

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(139): Could not remove or rename C:\ProgramData\Anaconda3\.condatmp\a2c29f57-bd05-4131-801a-d719d358920a.conda_trash.  Please remove this file manually (you may need to reboot to free file handles)

EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda3


R[write to console]: Warning:
R[write to console]:  package 'foreign' is in use and will not be installed

R[write to console]: Warning:
R[write to console]:  package 'survey' is in use and will not be installed

R[write to console]: Warning:
R[write to console]:  package 'tidyverse' is in use and will not be installed

R[write to console]: Warning:
R[write to console]:  package 'readr' is in use and will not be installed

R[write to console]: Warning:
R[write to console]:  package 'devtools' is in use and will not be installed

R[write to console]: Skipping install of 'MEPS' f

|==================================================================| 100% 182 MB


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(
R[write to console]: trying URL 'https://meps.ahrq.gov/data_files/pufs/h206bdat.zip'



MEPS_PrescribedMeds table created in DB
Could not delete dataframe from memory as MEPS_PrescribedMeds Dataframe does not exist


R[write to console]: Content type 'application/zip'
R[write to console]:  length 695865 bytes (679 KB)

R[write to console]: downloaded 679 KB


R[write to console]: trying URL 'https://meps.ahrq.gov/data_files/pufs/h209dat.zip'

R[write to console]: Content type 'application/zip'
R[write to console]:  length 11600848 bytes (11.1 MB)

R[write to console]: downloaded 11.1 MB




|==============================================                     | 68% 88 MB

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



MEPS_PatientDemographics table created in DB
Could not delete dataframe from memory as MEPS_PatientDemographics Dataframe does not exist
request sent to URL: https://download.nlm.nih.gov/rxnorm/RxNorm_full_prescribe_current.zip and returned HTTP status code of 200
RXNCONSO table created in DB
Could not delete dataframe from memory as RXNCONSO Dataframe does not exist
RXNREL table created in DB
Could not delete dataframe from memory as RXNREL Dataframe does not exist
RXNSAT table created in DB
Could not delete dataframe from memory as RXNSAT Dataframe does not exist
DB table MEPS has 0 records
DB table RXNCONSO has 227709 records
DB table RXNREL has 2469131 records
DB table RXNSAT has 3024797 records
Read joey_query.sql file as string
joey_query table created in DB
Could not delete dataframe from memory as joey_query Dataframe does not exist
